In [1]:
import pandas as pd
df=pd.read_csv('c:/data/iris/iris.csv')
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name,Species
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0


In [2]:
from sqlalchemy import create_engine
import cx_Oracle
#오라클 서버에 접속하기 위한 엔진 생성
engine=create_engine('oracle+cx_oracle://python:0000@xe')
conn=engine.connect()
# 데이터프레임의 내용을 테아블로 export
df.to_sql(name='iris', con=engine, if_exists='replace', index=False)

150

In [3]:
import cx_Oracle
conn = cx_Oracle.connect("python/0000@localhost:1521/xe")
cursor = conn.cursor()                      
cursor.execute('select * from ontime where rownum < 5')
rows = cursor.fetchall()         
for row in rows:
    print(row)
cursor.close()  
conn.close()

(2007, 1, 1, 1, '1939', 1920, '2146', 2135, 'WN', '998', 'N746SW', '127', '135', '107', '11', '19', 'TPA', 'BWI', '842', '3', '17', '0', None, '0', '0', '0', '0', '0', '0')
(2007, 1, 1, 1, '731', 730, '940', 940, 'WN', '2157', 'N689SW', '129', '130', '110', '0', '1', 'TPA', 'BWI', '842', '7', '12', '0', None, '0', '0', '0', '0', '0', '0')
(2007, 1, 1, 1, '1931', 1910, '2132', 2125, 'WN', '1806', 'N650SW', '121', '135', '108', '7', '21', 'TPA', 'CMH', '829', '2', '11', '0', None, '0', '0', '0', '0', '0', '0')
(2007, 1, 1, 1, '1120', 1110, '1314', 1325, 'WN', '2503', 'N744SW', '114', '135', '101', '-11', '10', 'TPA', 'CMH', '829', '3', '10', '0', None, '0', '0', '0', '0', '0', '0')


In [6]:
#항공사별 출발 지연 시간 평균(실행시간이 오래 걸림)
import cx_Oracle
conn = cx_Oracle.connect("python/0000@localhost:1521/xe")
cursor = conn.cursor()                      
cursor.execute('''
select uniquecarrier, round(avg(depdelay),2)
from ontime
group by uniquecarrier
''')    
rows=cursor.fetchall()         
for row in rows:
    print('{}\t{}'.format(row[0],row[1]))

cursor.close()
conn.close()

NW	8.76
XE	11.58
UA	13.57
F9	7.38
AA	14.3
FL	8.95
AQ	0.44
AS	10.56
OO	8.11
OH	12.13
CO	11.68
YV	11.86
HA	-0.88
WN	10.35
US	11.92
9E	8.83
MQ	12.64
EV	19.59
DL	7.75
B6	14.45


In [7]:
#시간대별( 10월, 월요일에 출발할 때 가장 좋은 시간대는? )
import cx_Oracle
conn=cx_Oracle.connect("python/0000@localhost:1521/xe")
cursor=conn.cursor()                      
cursor.execute('''
select *
from (
    select rownum as rn, A.*
    from (
        select CRSDepTime, avg(arrdelay) arrdelay
        from ontime
        where month=10 and dayofweek=1
        group by CRSDepTime
        order by arrdelay
    ) A
) where rn between 1 and 5
''')    
rows=cursor.fetchall()         
for row in rows:
    print(row)

cursor.close()
conn.close()

(1, 1924, -47)
(2, 2257, -32)
(3, 300, -29.4)
(4, 2244, -20)
(5, 1801, -18.6)
